### Imports

In [40]:
import pandas as pd
import numpy as np
np.set_printoptions(threshold=np.inf)
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import tree
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jabel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jabel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Lectura de Datos

In [41]:
# Recogemos	los datos de la base de datos en un dataframe
df = pd.read_csv('../data/train.csv')

### Limpieza de texto

In [72]:
# Función para limpiar el texto
def clean_text(text):
	
	# Estandarizamos caracteres extraños
	if(type(text) == str):
		text = text.replace('“', '"')
		text = text.replace('”', '"')
		text = text.replace('’', "'")
		text = text.replace('‘', "'")
		text = text.replace('`', "'")
		text = text.replace('´', "'")
		text = text.replace('–', '-')
		text = text.replace('−', '-')
		text = text.replace('…', '...')
		text = text.replace('—', '-')
		text = text.replace('•', '-')
		text = text.replace('·', '-')
	
	
    # Eliminar urls
	text = re.sub(r'http\S+', ' ', str(text))
	
 	# Eliminar emojis
	emoji_pattern = re.compile("["
			u"\U0001F600-\U0001F64F"
			u"\U0001F300-\U0001F5FF"
			u"\U0001F680-\U0001F6FF"
			u"\U0001F1E0-\U0001F1FF"
			u"\U00002702-\U000027B0"
			u"\U000024C2-\U0001F251"
			"]+", flags=re.UNICODE)
	text = emoji_pattern.sub(r'', text)
	
 	# Eliminar palabras con números
	text = re.sub(r'\w*\d\w*', ' ', text)
	
    # Eliminar palabras que no estén en latino
	text = re.sub(r'[^\x00-\x7F]+', ' ', str(text))
 
 	# Eliminar signos de puntuación
	text = re.sub(r'[^\w\s]', ' ', str(text))
 
	# Eliminar caracteres especiales
	text = re.sub(r'\W', ' ', str(text))
 
 	# Eliminar números
	text = re.sub(r'\d+', ' ', text)
 
    # Minusculas
	text = text.lower()
 
 	# Eliminar contracciones
	
	text = re.sub(r"ain\'t", " am not", text)
	text = re.sub(r"aren\'t", " are not", text)
	text = re.sub(r"can\'t", " can not", text)
	text = re.sub(r"can\'t've", " can not have", text)
	text = re.sub(r"\'cause", " because	", text)
	text = re.sub(r"could\'ve", " could have", text)
	text = re.sub(r"couldn\'t", " could not", text)
	text = re.sub(r"couldn\'t've", " could not have", text)
	text = re.sub(r"didn\'t", " did not", text)
	text = re.sub(r"doesn\'t", " does not", text)
	text = re.sub(r"don\'t", " do not", text)
	text = re.sub(r"hadn\'t", " had not", text)
	text = re.sub(r"hadn\'t've", " had not have", text)
	text = re.sub(r"hasn\'t", " has not", text)
	text = re.sub(r"haven\'t", " have not", text)
	text = re.sub(r"he\'d", " he had", text)
	text = re.sub(r"he\'d've", " he would have", text)
	text = re.sub(r"he\'ll", " he will", text)
	text = re.sub(r"he\'ll've", " he will have", text)
	text = re.sub(r"he\'s", " he is", text)
	text = re.sub(r"how\'d", " how did", text)
	text = re.sub(r"how\'d'y", " how do you", text)
	text = re.sub(r"how\'ll", " how will", text)
	text = re.sub(r"how\'s", " how is", text)
	text = re.sub(r"how\'d", " how did", text)
	text = re.sub(r"i\'d", " i had", text)
	text = re.sub(r"i\'d've", " i would have", text)
	text = re.sub(r"i\'ll", " i will", text)
	text = re.sub(r"i\'ll've", " i will have", text)
	text = re.sub(r"i\'m", " i am", text)
	text = re.sub(r"i\'ve", " i have", text)
	text = re.sub(r"i\'ll", " i will", text)
	text = re.sub(r"isn\'t", " is not", text)
	text = re.sub(r"it\'d", " it had", text)
	text = re.sub(r"it\'d've", " it would have", text)
	text = re.sub(r"it\'ll", " it will", text)
	text = re.sub(r"it\'ll've", " it will have", text)
	text = re.sub(r"it\'s", " it is", text)
	text = re.sub(r"let\'s", " let us", text)
	text = re.sub(r"ma\'am", " madam", text)
	text = re.sub(r"mayn\'t", " may not", text)
	text = re.sub(r"might\'ve", " might have", text)
	text = re.sub(r"mightn\'t", " might not", text)
	text = re.sub(r"mightn\'t've", " might not have", text)
	text = re.sub(r"mightn\'t", " might not", text)
	text = re.sub(r"must\'ve", " must have", text)
	text = re.sub(r"mustn\'t", " must not", text)
	text = re.sub(r"mustn\'t've", " must not have", text)
	text = re.sub(r"needn\'t", " need not", text)
	text = re.sub(r"needn\'t've", " need not have", text)
	text = re.sub(r"o\'clock", " of the clock", text)
	text = re.sub(r"oughtn\'t", " ought not", text)
	text = re.sub(r"oughtn\'t've", " ought not have", text)
	text = re.sub(r"shan\'t", " shall not", text)
	text = re.sub(r"sha\'n't", " shall not", text)
	text = re.sub(r"shan\'t've", " shall not have", text)
	text = re.sub(r"she\'d", " she had", text)
	text = re.sub(r"she\'d've", " she would have", text)
	text = re.sub(r"she\'ll", " she will", text)
	text = re.sub(r"she\'ll've", " she will have", text)
	text = re.sub(r"she\'s", " she is", text)
	text = re.sub(r"should\'ve", " should have", text)
	text = re.sub(r"shouldn\'t", " should not", text)
	text = re.sub(r"shouldn\'t've", " should not have", text)
	text = re.sub(r"so\'ve", " so have", text)
	text = re.sub(r"so\'s", " so is", text)
	text = re.sub(r"that\'d", " that would", text)
	text = re.sub(r"that\'d've", " that would have", text)
	text = re.sub(r"that\'s", " that is", text)
	text = re.sub(r"there\'d", " there had", text)
	text = re.sub(r"there\'d've", " there would have", text)
	text = re.sub(r"there\'s", " there is", text)
	text = re.sub(r"they\'d", " they had", text)
	text = re.sub(r"they\'d've", " they would have", text)
	text = re.sub(r"they\'ll", " they will", text)
	text = re.sub(r"they\'ll've", " they will have", text)
	text = re.sub(r"they\'re", " they are", text)
	text = re.sub(r"they\'ve", " they have", text)
	text = re.sub(r"to\'ve", " to have", text)
	text = re.sub(r"wasn\'t", " was not", text)
	text = re.sub(r"we\'d", " we had", text)
	text = re.sub(r"we\'d\'ve", " we would have", text)
	text = re.sub(r"we\'ll", " we will", text)
	text = re.sub(r"we\'ll\'ve", " we will have", text)
	text = re.sub(r"we\'re", " we are", text)
	text = re.sub(r"we\'ve", " we have", text)
	text = re.sub(r"weren\'t", " were not", text)
	text = re.sub(r"what\'ll", " what will", text)
	text = re.sub(r"what\'ll\'ve", " what will have", text)
	text = re.sub(r"what\'re", " what are", text)
	text = re.sub(r"what\'s", " what is", text)
	text = re.sub(r"what\'ve", " what have", text)
	text = re.sub(r"when\'s", " when is", text)
	text = re.sub(r"when\'ve", " when have", text)
	text = re.sub(r"where\'d", " where did", text)
	text = re.sub(r"where\'s", " where is", text)
	text = re.sub(r"where\'ve", " where have", text)
	text = re.sub(r"who\'ll", " who will", text)
	text = re.sub(r"who\'ll\'ve", " who will have", text)
	text = re.sub(r"who\'s", " who is", text)
	text = re.sub(r"who\'ve", " who have", text)
	text = re.sub(r"why\'s", " why is", text)
	text = re.sub(r"why\'ve", " why have", text)
	text = re.sub(r"will\'ve", " will have", text)
	text = re.sub(r"won\'t", " will not", text)
	text = re.sub(r"won\'t\'ve", " will not have", text)
	text = re.sub(r"would\'ve", " would have", text)
	text = re.sub(r"wouldn\'t", " would not", text)
	text = re.sub(r"wouldn\'t\'ve", " would not have", text)
	text = re.sub(r"y\'all", " you all", text)
	text = re.sub(r"y\'all\'d\'ve", " you all would have", text)
	text = re.sub(r"y\'all\'d\'ve", " you all would have", text)
	text = re.sub(r"y\'all\'re", " you all are", text)
	text = re.sub(r"y\'all\'ve", " you all have", text)
	text = re.sub(r"you\'d\'ve", " you would have", text)
	text = re.sub(r"you\'ll", " you will", text)
	text = re.sub(r"you\'ll\'ve", " you will have", text)
	text = re.sub(r"you\'re", " you are", text)
	text = re.sub(r"you\'ve", " you have", text)
 
	# Eliminar espacios en blanco
	text = re.sub(r'\s{2,}', ' ', text)
 
	# Eliminar stopwords
	stop_words = set(stopwords.words('english'))
	text = text.split()
	text = [w for w in text if not w in stop_words]
	text = " ".join(text)
 
	# Stemming
	text = text.split()
	stemmer = SnowballStemmer('english')
	stemmed_words = [stemmer.stem(word) for word in text]
	text = " ".join(stemmed_words)
 	
	return text

In [73]:
# Texto limpio en un nuevo dataframe
df_clean = pd.DataFrame()
df_clean['text'] = df['title'] + ' ' + df['text']
df_clean['text'] = df_clean['text'].apply(clean_text)
df_clean['label'] = df['label']

### Tokenización

In [75]:
# Tokenizamos el texto
df_clean_tokens = pd.DataFrame()
df_clean_tokens['text'] = df_clean['text'].apply(nltk.word_tokenize)
df_clean_tokens['label'] = df_clean['label']

In [77]:
# Si label es 0, incluimos en text True, False si es 1
for i in range(len(df_clean_tokens['label'])):
	if df_clean_tokens['label'][i] == 0:
		df_clean_tokens['text'].iloc[i].append("TRUE")
	else:
		df_clean_tokens['text'].iloc[i].append("FALSE")
df_clean_tokens.drop('label', 1)	

C:\Users\jabel\AppData\Local\Temp\ipykernel_14632\1837252537.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_clean_tokens.drop('label', 1)


,text
0,"[hous, dem, aid, even, see, comey, letter, jas..."
1,"[flynn, hillari, clinton, big, woman, campus, ..."
2,"[truth, might, get, fire, truth, might, get, f..."
3,"[civilian, kill, singl, us, airstrik, identifi..."
4,"[iranian, woman, jail, fiction, unpublish, sto..."
...,...
20795,"[rapper, trump, poster, child, white, supremac..."
20796,"[n, f, l, playoff, schedul, matchup, odd, new,..."
20797,"[maci, said, receiv, takeov, approach, hudson,..."
20798,"[nato, russia, hold, parallel, exercis, balkan..."


### Split train/test

In [79]:
# Dividimos los datos en train y test
p_train = 0.80

df_clean_tokens['is_train'] = np.random.uniform(0, 1, len(df)) <= p_train
train = df_clean_tokens[df_clean_tokens['is_train'] == True]
test =  df_clean_tokens[df_clean_tokens['is_train'] == False]
df_clean_tokens = df_clean_tokens.drop('is_train', 1)

print("Ejemplos usados para entrenar: ", len(train))
print("Ejemplos usados para test: ", len(test))

Ejemplos usados para entrenar:  16586
Ejemplos usados para test:  4214


C:\Users\jabel\AppData\Local\Temp\ipykernel_14632\2623265399.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_clean_tokens = df_clean_tokens.drop('is_train', 1)


### Apriori

In [80]:
# Preparamos los datos para procesar Apriori
te = TransactionEncoder()
te_ary = te.fit(train['text']).transform(train['text'])
df_apriori = pd.DataFrame(te_ary, columns=te.columns_)
df_apriori

,False,True,_,__,___,____,_____,______,_______,________,...,zx,zxycmwjdxj,zy,zyaira,zygankow,zygar,zylinderkopfdichtung,zytsov,zyuganov,zzzzs
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16581,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
16582,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
16583,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
16584,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [95]:
# Conjuntos de items frecuentes
itemsets_frecuentes = apriori(df_apriori, min_support=0.2, use_colnames=True)
itemsets_frecuentes

,support,itemsets
0,0.499216,(False)
1,0.500784,(True)
2,0.302363,(accord)
3,0.267274,(ad)
4,0.534246,(also)
...,...,...
4410,0.224044,"(year, one, said, york, time, new)"
4411,0.203485,"(year, one, york, time, would, new)"
4412,0.211685,"(year, said, york, time, would, new)"
4413,0.203364,"(one, True, said, york, time, would, new)"


In [96]:
# Reglas de asociación apriori
reglas = association_rules(itemsets_frecuentes, metric="confidence", min_threshold=0.2)
reglas

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(False),(also),0.499216,0.534246,0.202279,0.405193,0.758440,-0.064425,0.783035
1,(also),(False),0.534246,0.499216,0.202279,0.378625,0.758440,-0.064425,0.805929
2,(False),(like),0.499216,0.528578,0.222778,0.446256,0.844257,-0.041097,0.851335
3,(like),(False),0.528578,0.499216,0.222778,0.421467,0.844257,-0.041097,0.865610
4,(False),(one),0.499216,0.610093,0.251236,0.503261,0.824892,-0.053332,0.784933
...,...,...,...,...,...,...,...,...,...
42053,(True),"(year, one, said, york, time, new)",0.500784,0.224044,0.203666,0.406694,1.815238,0.091468,1.307851
42054,(said),"(year, one, True, york, time, new)",0.619197,0.218075,0.203666,0.328919,1.508281,0.068634,1.165172
42055,(york),"(year, one, True, said, time, new)",0.392560,0.211926,0.203666,0.518814,2.448095,0.120472,1.637776
42056,(time),"(year, one, True, said, york, new)",0.619740,0.205052,0.203666,0.328631,1.602669,0.076587,1.184070


In [99]:
# Escogemos las reglas que contienen solamente True o solamente False en el consecuente
reglas_VoF = reglas[(reglas['consequents'] == frozenset({'TRUE'})) | (reglas['consequents'] == frozenset({'FALSE'}))]
reglas_VoF.sort_values(by=['confidence'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
39455,"(said, york, time, mr, new)",(True),0.232907,0.500784,0.226637,0.973078,1.943110,0.110001,18.543002
30170,"(said, york, time, mr)",(True),0.232907,0.500784,0.226637,0.973078,1.943110,0.110001,18.543002
13430,"(said, york, mr)",(True),0.233209,0.500784,0.226818,0.972596,1.942147,0.110031,18.216682
30080,"(said, york, mr, new)",(True),0.233209,0.500784,0.226818,0.972596,1.942147,0.110031,18.216682
13444,"(york, time, mr)",(True),0.246051,0.500784,0.237369,0.964715,1.926409,0.114151,14.147926
...,...,...,...,...,...,...,...,...,...
16,(year),(False),0.532437,0.499216,0.209936,0.394293,0.789824,-0.055865,0.826775
15,(would),(False),0.539612,0.499216,0.209454,0.388156,0.777532,-0.059929,0.818483
1,(also),(False),0.534246,0.499216,0.202279,0.378625,0.758440,-0.064425,0.805929
13,(time),(False),0.619740,0.499216,0.231400,0.373383,0.747938,-0.077984,0.799186
